In [ ]:
import openeo
import json
import geopandas as gpd
from shapely.geometry import mapping  # Import mapping to convert geometry to GeoJSON

In [ ]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

In [ ]:
geo_df = gpd.read_file("NUTS_RG_20M_2024_4326.geojson")
geo_df_italy = geo_df[geo_df["CNTR_CODE"] == "IT"][:107]  #only nuts3 
geo_df_belgium = geo_df[geo_df["CNTR_CODE"] == "BE"][:44] #only nuts3

In [ ]:
nuts_3_italy =list(geo_df_italy["NUTS_NAME"])
nuts_3_belgium = list(geo_df_belgium["NUTS_NAME"])

In [ ]:
def extract_aoi(row: gpd.GeoSeries):
    """Extracts geometry from a GeoSeries row and converts it into a dictionary."""
    return mapping(row.geometry)  # Convert geometry to GeoJSON format

def get_data(geo_df: gpd.GeoDataFrame, connection, nuts_3_name, folder):
    """Loads Sentinel-2 data, computes NDVI, aggregates it over the area, and saves the result."""
    i = 0
    for _, row in geo_df.iterrows():
        aoi = extract_aoi(row)  # Corrected extract_aoi function
        
        ndvi_cube = connection.load_collection(
            "SENTINEL2_L2A",
            temporal_extent=["2024-01-01", "2024-12-31"],
            bands=["B04", "B08"],
            spatial_extent=aoi
        )
        
        red = ndvi_cube.band("B04")
        nir = ndvi_cube.band("B08")
        ndvi = (nir - red) / (nir + red)
        ndvi_aggregated = ndvi.aggregate_spatial(geometries=aoi, reducer="mean")
        #ndvi_aggregated = ndvi_aggregated.save_result(format="NetCDF")
        # Create and start job
        print(f"{nuts_3_name[i]} started")
        job = ndvi_aggregated.execute_batch(title = f"{nuts_3_name[i]}_ndvi_ts", outputfile=f"data/NDVI_nc/{folder}/{nuts_3_name[i]}_ndvi_ts.nc")
        print(f"{nuts_3_name[i]} finished")
        i +=1

In [ ]:
get_data(geo_df_italy, connection, nuts_3_italy, folder = "Italia")

In [ ]:
get_data(geo_df_belgium, connection, nuts_3_belgium, folder = "Belgio")